### Setup

In [1]:
import os, sys, importlib, time
from dotenv import load_dotenv

# Get directories first to find .env file
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
env_path = os.path.join(parent_dir, '.env')

# Load .env file
load_dotenv(env_path, override=True)

# Get timezone from .env with default fallback
timezone = os.getenv('TZ', 'America/New_York')
os.environ["TZ"] = timezone
time.tzset()

# Get additional directories
rag_dir = os.path.join(parent_dir, 'RAG')
print(f"Current directory: {current_dir}")
print(f"Parent directory: {parent_dir}")
print(f"Parent dir exists: {os.path.exists(parent_dir)}")
print(f"RAG dir exists: {os.path.exists(rag_dir)}")
print(f"KnowledgeGraph exists: {os.path.exists(os.path.join(parent_dir, 'KnowledgeGraph'))}")

# Add paths to sys.path BEFORE importing modules
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
if rag_dir not in sys.path:
    sys.path.insert(0, rag_dir)
    
# Import modules after paths are set up
from KnowledgeGraph.config import load_neo4j_graph
import RAG.VectorRAG as VectorRAG
import RAG.GraphRAG as GraphRAG
import RAG.batch_query as batch_query

# make sure AB Testing is on sys.path
test_dir = os.path.join(parent_dir, "AB Testing")
if test_dir not in sys.path:
    sys.path.insert(0, test_dir)

# import/reload mark_answers, then get the function
import mark_answers as mark_answers_module
importlib.reload(mark_answers_module)
mark_answers = mark_answers_module.mark_answers

# Reload other modules to pick up changes
importlib.reload(VectorRAG)
importlib.reload(GraphRAG)
importlib.reload(batch_query)

# TODO Remove this
from RAG.GraphRAG import query_graph_rag

# TODO understand this
graph, openAI_api, openAI_endpoint, openAI_model = load_neo4j_graph()

Current directory: /mnt/c/Users/sepeh/OneDrive/Documents/Git/GraphRag_Developer_Challenge3/Main Functions
Parent directory: /mnt/c/Users/sepeh/OneDrive/Documents/Git/GraphRag_Developer_Challenge3
Parent dir exists: True
RAG dir exists: True
KnowledgeGraph exists: True


/home/sepeh/.cache/pypoetry/virtualenvs/graphrag-developer-challenge3-4suqu0LP-py3.12/lib/python3.12/site-packages/neo4j/_sync/driver.py:547: DeprecationWarning: Relying on Driver's destructor to close the session is deprecated. Please make sure to close the session. Use it as a context (`with` statement) or make sure to call `.close()` explicitly. Future versions of the driver will not close drivers automatically.
  _deprecation_warn(


### GraphRag Query

In [ ]:
# Supplement trick question
GraphRAG.query_graph_rag("What is the company name listed for Tender RFP-2113556?")

/home/sepeh/.cache/pypoetry/virtualenvs/graphrag-developer-challenge3-4suqu0LP-py3.12/lib/python3.12/site-packages/neo4j_graphrag/generation/graphrag.py:120: DeprecationWarning: The default value of 'return_context' will change from 'False' to 'True' in a future version.
  warnings.warn(
/home/sepeh/.cache/pypoetry/virtualenvs/graphrag-developer-challenge3-4suqu0LP-py3.12/lib/python3.12/site-packages/neo4j_graphrag/retrievers/vector.py:201: DeprecationWarning: The default returned 'id' field in the search results will be removed. Please switch to using 'elementId' instead.
  search_query, search_params = get_search_query(


'Kuwait Oil Company'

In [4]:
# Supplement trick question
GraphRAG.query_graph_rag("On what date was the earlier announcement for Tender RFP-2113556 published?")

/home/sepeh/.cache/pypoetry/virtualenvs/graphrag-developer-challenge3-4suqu0LP-py3.12/lib/python3.12/site-packages/neo4j_graphrag/generation/graphrag.py:120: DeprecationWarning: The default value of 'return_context' will change from 'False' to 'True' in a future version.
  warnings.warn(
/home/sepeh/.cache/pypoetry/virtualenvs/graphrag-developer-challenge3-4suqu0LP-py3.12/lib/python3.12/site-packages/neo4j_graphrag/retrievers/vector.py:201: DeprecationWarning: The default returned 'id' field in the search results will be removed. Please switch to using 'elementId' instead.
  search_query, search_params = get_search_query(


'I can’t find Tender RFP-2113556 in the provided context, so the earlier announcement date isn’t available here. If you share the specific notice or issue number from Kuwait Today (Al-Kuwait Al-Youm), I can extract the date.'

In [2]:
batch_query.batch_query_graph_rag("AB Testing/questions_long.json", max_workers=60)

/home/sepeh/.cache/pypoetry/virtualenvs/graphrag-developer-challenge3-4suqu0LP-py3.12/lib/python3.12/site-packages/neo4j_graphrag/generation/graphrag.py:120: DeprecationWarning: The default value of 'return_context' will change from 'False' to 'True' in a future version.
  warnings.warn(
/home/sepeh/.cache/pypoetry/virtualenvs/graphrag-developer-challenge3-4suqu0LP-py3.12/lib/python3.12/site-packages/neo4j_graphrag/generation/graphrag.py:120: DeprecationWarning: The default value of 'return_context' will change from 'False' to 'True' in a future version.
  warnings.warn(
/home/sepeh/.cache/pypoetry/virtualenvs/graphrag-developer-challenge3-4suqu0LP-py3.12/lib/python3.12/site-packages/neo4j_graphrag/generation/graphrag.py:120: DeprecationWarning: The default value of 'return_context' will change from 'False' to 'True' in a future version.
  warnings.warn(
/home/sepeh/.cache/pypoetry/virtualenvs/graphrag-developer-challenge3-4suqu0LP-py3.12/lib/python3.12/site-packages/neo4j_graphrag/gen

Processed 71 queries with 60 workers
Results saved to: /mnt/c/Users/sepeh/OneDrive/Documents/Git/GraphRag_Developer_Challenge3/AB Testing/Output Answers/answers_20251102_122645.json


[{'question': 'What is the time period to submit an objection for company registration changes?',
  'answer': '- Most company registration changes (e.g., amending the legal entity, dissolution/liquidation) handled by the Partnerships Department: 30 days from the publication date to file an objection with the General Court (Court of First Instance).\n- Converting an individual license to a company (handled by the Companies Department): 15 days from the publication date to submit a written objection to the department, accompanied by a debt instrument.'},
 {'question': 'Where should objections to company registration be submitted?',
  'answer': 'To the General Court (Court of First Instance, “المحكمة الكلية”).'},
 {'question': 'What types of legal entities are mentioned in company registration announcements?',
  'answer': '- One-Person Company (Single Person Company, SPC) — شركة الشخص الواحد (ش.ش.و)\n- Limited Liability Company (LLC) — شركة ذات مسؤولية محدودة\n- General Partnership — شركة

In [2]:
# Supplement trick questions
batch_query.batch_query_graph_rag("AB Testing/questions_supplement.json", max_workers=60)

/home/sepeh/.cache/pypoetry/virtualenvs/graphrag-developer-challenge3-4suqu0LP-py3.12/lib/python3.12/site-packages/neo4j_graphrag/generation/graphrag.py:120: DeprecationWarning: The default value of 'return_context' will change from 'False' to 'True' in a future version.
  warnings.warn(
/home/sepeh/.cache/pypoetry/virtualenvs/graphrag-developer-challenge3-4suqu0LP-py3.12/lib/python3.12/site-packages/neo4j_graphrag/generation/graphrag.py:120: DeprecationWarning: The default value of 'return_context' will change from 'False' to 'True' in a future version.
  warnings.warn(
/home/sepeh/.cache/pypoetry/virtualenvs/graphrag-developer-challenge3-4suqu0LP-py3.12/lib/python3.12/site-packages/neo4j_graphrag/generation/graphrag.py:120: DeprecationWarning: The default value of 'return_context' will change from 'False' to 'True' in a future version.
  warnings.warn(
/home/sepeh/.cache/pypoetry/virtualenvs/graphrag-developer-challenge3-4suqu0LP-py3.12/lib/python3.12/site-packages/neo4j_graphrag/gen

Processed 8 queries with 60 workers
Results saved to: /mnt/c/Users/sepeh/OneDrive/Documents/Git/GraphRag_Developer_Challenge3/AB Testing/Output Answers/answers_20251102_113359.json


[{'question': 'What company name is listed for Tender RFP-2113556?',
  'answer': 'I can’t find Tender RFP-2113556 in the provided context, so the company name isn’t available here. If you share the relevant notice or issue/page, I can look it up.'},
 {'question': 'What legal entity is stated for Tender RFP-2113556 in the Ministry of Commerce and Industry announcement?',
  'answer': 'It isn’t in the provided context. The excerpts don’t mention Tender RFP-2113556, so no legal entity is stated here.'},
 {'question': 'On what date was the earlier announcement for Tender RFP-2113556 published?',
  'answer': 'The provided context does not mention Tender RFP-2113556, so I can’t determine the earlier announcement date from this material. If you share the relevant announcement (or the Kuwait Today issue number referencing RFP-2113556), I can extract the date.'},
 {'question': 'Which company name should be used going forward for Tender RFP-2113556?',
  'answer': 'FUTURE SERVICES GENERAL TRADING 

### Mark Answers Supplement

In [ ]:
# Supplement trick questions
mark_answers(answer_key_path = "AB Testing/answer_key_supplement.json", max_workers = 60)


TEST RESULTS

Configuration:
  top_k: 30
  MMR_k: 15
  MMR_LAMBDA: 0.4
  ALWAYS_KEEP_TOP: 3
  neighbor_window: 1

Results:
  Average F1: 16.7% (0.167)

Card-first gate:
  CARD_FIRST_ENABLED: True
  ALLOW_DIRECT_ANSWER: True



{'saved_at': '2025-11-02T11:52:58',
 'expected_file': '/mnt/c/Users/sepeh/OneDrive/Documents/Git/GraphRag_Developer_Challenge3/AB Testing/answer_key_supplement.json',
 'answers_file': '/mnt/c/Users/sepeh/OneDrive/Documents/Git/GraphRag_Developer_Challenge3/AB Testing/Output Answers/answers_20251102_113359.json',
 'config': {'top_k': 30,
  'MMR_k': 15,
  'MMR_LAMBDA': 0.4,
  'ALWAYS_KEEP_TOP': 3,
  'neighbor_window': 1,
  'CARD_FIRST_ENABLED': True,
  'ALLOW_DIRECT_ANSWER': True},
 'overall': {'evaluated': 8.0,
  'average_f1': 0.16675,
  'average_f1_percent': 16.675},
 'questions': [{'question': 'What company name is listed for Tender RFP-2113556?',
   'expected': 'FUTURE SERVICES GENERAL TRADING AND CONTRACTING COMPANY W.L.L. [1]\n\nReferences\n1. Kuwait Official Gazette 2025-05-04 (Supplement), Lines 15490–15492.',
   'answer': 'I can’t find Tender RFP-2113556 in the provided context, so the company name isn’t available here. If you share the relevant notice or issue/page, I can look 

### Mark Answers Long

In [3]:
mark_answers(answer_key_path = "AB Testing/answer_key_long.json", max_workers = 60)


TEST RESULTS

Configuration:
  top_k: 30
  MMR_k: 15
  MMR_LAMBDA: 0.4
  ALWAYS_KEEP_TOP: 3
  neighbor_window: 1

Results:
  Average F1: 49.6% (0.496)

Card-first gate:
  CARD_FIRST_ENABLED: True
  ALLOW_DIRECT_ANSWER: True



{'saved_at': '2025-11-02T12:27:07',
 'expected_file': '/mnt/c/Users/sepeh/OneDrive/Documents/Git/GraphRag_Developer_Challenge3/AB Testing/answer_key_long.json',
 'answers_file': '/mnt/c/Users/sepeh/OneDrive/Documents/Git/GraphRag_Developer_Challenge3/AB Testing/Output Answers/answers_20251102_122645.json',
 'config': {'top_k': 30,
  'MMR_k': 15,
  'MMR_LAMBDA': 0.4,
  'ALWAYS_KEEP_TOP': 3,
  'neighbor_window': 1,
  'CARD_FIRST_ENABLED': True,
  'ALLOW_DIRECT_ANSWER': True},
 'overall': {'evaluated': 71.0,
  'average_f1': 0.4958591549295775,
  'average_f1_percent': 49.58591549295775},
 'questions': [{'question': 'What is the time period to submit an objection for company registration changes?',
   'expected': 'Thirty days from the date of publication in the official gazette [1].\n\nReferences\n1. Kuwait Official Gazette 2025-04-27, Lines 5228-5327.',
   'answer': '- Most company registration changes (e.g., amending the legal entity, dissolution/liquidation) handled by the Partnerships D